In [1]:
import geopandas as gpd
from shapely.geometry import box
from osgeo import gdal
from copy import deepcopy
import os, math
import numpy as np
import requests
import pandas as pd
from urllib.parse import urlparse
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import zipfile
import warnings

In [2]:
os.chdir(r"C:\Users\andim\Documents\AMANALCO"

In [3]:
LIMITE_MUNICIPAL = gpd.read_file("C:/Users/andim/Documents/MAPA BASE/INSUMOS BASE/00mun.shp").to_crs(epsg=6372)
print ("Carga completa")

def buscar_municipio(nom_ent, nomgeo):
    resultado = LIMITE_MUNICIPAL[(LIMITE_MUNICIPAL['NOM_ENT'] == nom_ent) & (LIMITE_MUNICIPAL['NOMGEO'] == nomgeo)]    
    if not resultado.empty:
        resultado = resultado[['CVE_ENT', 'NOM_ENT', 'NOMGEO', 'CVEGEO']]
        print(f"Municipio encontrado:\n{resultado}")
    else:
        print(f"No se encontró el municipio con ESTADO '{nom_ent}' y MUNICIPIO '{nomgeo}'.")
    
    return resultado

Carga completa


In [4]:
nom_ent = 'Ciudad de México'
nomgeo = 'Cuauhtémoc'
resultado = buscar_municipio(nom_ent, nomgeo)

Municipio encontrado:
    CVE_ENT           NOM_ENT      NOMGEO CVEGEO
286      09  Ciudad de México  Cuauhtémoc  09015


In [5]:
cvegeo = '09015' #CAMBIAR
cve_ent = '09' #CAMBIAR
municipio = LIMITE_MUNICIPAL[LIMITE_MUNICIPAL['CVEGEO'] == cvegeo]
bounds = municipio.total_bounds
bounding_box = box(bounds[0], bounds[1], bounds[2], bounds[3])
buffer = bounding_box.buffer(2000) #área del buffer par recortar
#zona_corte = gpd.GeoDataFrame(geometry=[buffer], crs=LIMITE_MUNICIPAL.crs).to_crs(epsg=6372) #Cortar con base en un buffer
zona_corte = gpd.GeoDataFrame(geometry=[bounding_box], crs=LIMITE_MUNICIPAL.crs).to_crs(epsg=6372) #Cortar con base en las coordenadas límites
print(zona_corte)

                                            geometry
0  POLYGON ((2801250.362 825359.340, 2801250.362 ...


In [6]:
zona_corte.to_file(f"C{cvegeo}_ZONACORTE_EP_2025.shp")

In [7]:
cartas = gpd.read_file(r'C:\Users\brenp\Documents\GitHub\DESGARGAS_INEGI\scrips\INSUMOS BASE\div50k_lcc.parquet').to_crs(epsg=6372)
claves = cartas['CLAVE50K'].unique()
claves_en_zona_corte = gpd.sjoin(cartas, zona_corte, how='inner', predicate='intersects')
claves_cartas_topograficas = claves_en_zona_corte['CLAVE50K'].unique()
print(claves_cartas_topograficas)

['E14A39']


In [8]:
# Ruta al WebDriver de Chrome
webdriver_path = r'C:\Users\andim\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe'
options = webdriver.ChromeOptions()
#options.add_argument('--headless')
options.add_argument('headless=False')
options.add_argument('--disable-gpu')
service = Service(webdriver_path)
driver = webdriver.Chrome(service=service, options=options)

In [9]:
url = 'https://www.inegi.org.mx/programas/topografia/50000/#descargas'
driver.get(url)

In [10]:
wait = WebDriverWait(driver, 100)
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input#bcm_resultBusqueda3_input_search')))
clave_enlace_list = []
def buscar_clave(clave):
    search_box = driver.find_element(By.CSS_SELECTOR, 'input#bcm_resultBusqueda3_input_search')
    search_button = driver.find_element(By.CSS_SELECTOR, 'button#bcm_buscar_btn_resultBusqueda3')
    driver.execute_script("arguments[0].scrollIntoView();", search_box)
    driver.execute_script("arguments[0].scrollIntoView();", search_button)
    search_box.clear()
    search_box.send_keys(clave)
    WebDriverWait(driver, 100).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button#bcm_buscar_btn_resultBusqueda3')))
    driver.execute_script("arguments[0].click();", search_button)
    WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'tbody')))

def procesar_filas(clave_buscar):
    rows = driver.find_elements(By.CSS_SELECTOR, 'tbody tr')
    for row in rows:
        try:
            clave_element = row.find_element(By.CSS_SELECTOR, 'td:nth-child(1) .tablesaw-cell-content')
            clave = clave_element.text.strip()
            edicion_element = row.find_element(By.CSS_SELECTOR, 'td:nth-child(4) .tablesaw-cell-content')
            edicion = edicion_element.text.strip()
            enlace_descarga = None
            for i in [8, 9, 10]:
                try:
                    tipo_archivo_element = row.find_element(By.CSS_SELECTOR, f'td:nth-child({i}) img[src="/img/ico/ico_shp.png"]')
                    if tipo_archivo_element:
                        enlace_element = row.find_element(By.CSS_SELECTOR, f'td:nth-child({i}) a[href$=".zip"]')
                        enlace_descarga = enlace_element.get_attribute('href')
                        break
                except Exception:
                    continue
            if enlace_descarga:
                clave_enlace_list.append((clave, edicion, enlace_descarga))
        
        except Exception:
            continue
for clave_buscar in claves_cartas_topograficas:
    buscar_clave(clave_buscar)
    time.sleep(10)
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        procesar_filas(clave_buscar)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
if clave_enlace_list:
    for clave, edicion, enlace in clave_enlace_list:
        print(f"Clave: {clave} - Edición: {edicion} - Enlace de descarga: {enlace}")
else:
    print("No se encontraron resultados para las claves buscadas")

driver.quit()

Clave: E14A39 - Edición:  - Enlace de descarga: https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/imagen_cartografica/1_50_000/889463854135_s.zip
Clave: E14A39 - Edición: 2019 - Enlace de descarga: https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/imagen_cartografica/1_50_000/889463833406_s.zip
Clave: E14A39 - Edición: III - Enlace de descarga: https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/imagen_cartografica/1_50_000/702825266721_s.zip
Clave: E14A39 - Edición: II - Enlace de descarga: https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/imagen_cartografica/1_50_000/702825003701_s.zip


In [11]:
def descargar_archivo(url, carpeta_destino, nombre_archivo):
    respuesta = requests.get(url, stream=True)
    ruta_archivo = os.path.join(carpeta_destino, nombre_archivo)
    
    with open(ruta_archivo, 'wb') as archivo:
        for chunk in respuesta.iter_content(chunk_size=32768):
            if chunk:
                archivo.write(chunk)
    
    print(f"Descargado: {ruta_archivo}")

edicion_deseada = "III" #SE PUEDE CAMBIAR POR LA SERIE O AÑO DE INTERÉS
carpeta_descargas = "CARTAS TOPOGRAFICAS" #CAMBIAR
os.makedirs(carpeta_descargas, exist_ok=True)

for clave, edicion, enlace in clave_enlace_list:
    if edicion == edicion_deseada:
        nombre_archivo = f"{clave}_{edicion}.zip"
        descargar_archivo(enlace, carpeta_descargas, nombre_archivo)

print("Descarga completa.")

Descargado: CARTAS TOPOGRAFICAS\E14A39_III.zip
Descarga completa.


In [12]:
carpeta_zip = 'CARTAS TOPOGRAFICAS' #CAMBIAR
os.makedirs(carpeta_zip, exist_ok=True)

for archivo_zip in os.listdir(carpeta_zip):
    if archivo_zip.endswith('.zip'):
        ruta_zip = os.path.join(carpeta_zip, archivo_zip)
        nombre = os.path.splitext(archivo_zip)[0]
        carpeta = os.path.join(carpeta_zip, nombre)
        os.makedirs(carpeta, exist_ok=True)
        with zipfile.ZipFile(ruta_zip, 'r') as archivo_zip_obj:
            archivo_zip_obj.extractall(carpeta)
            print(f"Descomprimido: {ruta_zip} en {carpeta}")

print("Descompresión completa.")

Descomprimido: CARTAS TOPOGRAFICAS\E14A39_III.zip en CARTAS TOPOGRAFICAS\E14A39_III
Descompresión completa.


In [13]:
# Buscar archivos .shp en una carpeta
def buscar_archivos_shp(carpeta):
    archivos_shp = []    
    for raiz, directorios, archivos in os.walk(carpeta):
        for archivo in archivos:
            if archivo.endswith('.shp'):
                ruta_archivo = os.path.join(raiz, archivo)
                archivos_shp.append(ruta_archivo)    
    return archivos_shp
    
for carpeta in os.listdir(carpeta_zip):
    ruta_carpeta = os.path.join(carpeta_zip, carpeta)    
    if os.path.isdir(ruta_carpeta):
        ruta_conjunto_datos = os.path.join(ruta_carpeta, 'conjunto_de_datos')        
        if os.path.isdir(ruta_conjunto_datos):
            archivos_shp = buscar_archivos_shp(ruta_conjunto_datos)            
            if archivos_shp:
                print(f"Archivos .shp encontrados en {ruta_conjunto_datos}:")
                for archivo in archivos_shp:
                    print(f"  - {archivo}")
            else:
                print(f"No se encontraron archivos .shp en {ruta_conjunto_datos}.")

Archivos .shp encontrados en CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos:
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\acueducto50_l.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\aeropuerto50_a.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\area_cultiv50_a.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\area_ver_ur50_a.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\ban_mat50_p.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\bordo50_l.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\calle50_l.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\camino50_l.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\canal50_a.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\canal50_l.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\carretera50_l.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\caseta_peaj50_p.shp
  - CARTAS TOPOGRAFICAS\E14A39_III\conjunto_de_datos\cementerio50_a.shp
  - CARTAS 

In [14]:
nombres_archivos = ['acueducto50_l.shp', 'canal50_l.shp', 'corriente_ag50_l.shp','cuerpo_agua50_a.shp', 'curva_nivel50_l.shp', 'linea_trans50_l.shp','canal50_a.shp', 'conducto50_l.shp', 'via_ferrea50_l.shp','aeropuerto50_a.shp']
def buscar_archivos(carpeta, nombres_archivos):
    archivos_encontrados = {nombre: [] for nombre in nombres_archivos}
    for raiz, directorios, archivos in os.walk(carpeta):
        for archivo in archivos:
            if archivo in nombres_archivos:
                archivos_encontrados[archivo].append(os.path.join(raiz, archivo))
    return archivos_encontrados
carpeta_zip = 'CARTAS TOPOGRAFICAS'
gdfs_dict = {nombre: [] for nombre in nombres_archivos}
for carpeta in os.listdir(carpeta_zip):
    ruta = os.path.join(carpeta_zip, carpeta)
    if os.path.isdir(ruta):
        ruta_conjunto_de_datos = os.path.join(ruta, 'conjunto_de_datos')
        if os.path.isdir(ruta_conjunto_de_datos):
            archivos_encontrados = buscar_archivos(ruta_conjunto_de_datos, nombres_archivos)            
            for nombre_archivo, rutas_archivos in archivos_encontrados.items():
                for ruta_archivo in rutas_archivos:
                    gdf = gpd.read_file(ruta_archivo)
                    gdf = gdf.to_crs(epsg=6372)
                    gdf['Cve_Ent'] = cve_ent
                    gdf['Cve_MunC'] = cvegeo
                    campos_a_conservar = [campo for campo in ['geografico', 'elevacion', 'nombre'] if campo in gdf.columns]
                    gdf = gdf[['geometry', 'Cve_Ent', 'Cve_MunC'] + campos_a_conservar]
                    gdfs_dict[nombre_archivo].append(gdf)
                    
for nombre_archivo, gdfs in gdfs_dict.items():
    if gdfs:
        gdf_unido = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
        if gdf_unido.crs != 'EPSG:6372':
            gdf_unido = gdf_unido.to_crs(epsg=6372)        
        gdf_cortado = gpd.overlay(gdf_unido, zona_corte, how='intersection')        
        if not gdf_cortado.empty:
            nombre_archivo_mayus = nombre_archivo.upper()
            nombre_salida = f"C{cvegeo}_{os.path.splitext(nombre_archivo_mayus)[0]}_INEGI_SERIE_III.shp"
            ruta_salida = os.path.join(carpeta_zip, nombre_salida)
            gdf_cortado.to_file(ruta_salida)
            print(f"Archivo unido y cortado guardado en: {ruta_salida}")
        else:
            print(f"El GeoDataFrame resultante de {nombre_archivo} está vacío después del corte.")
print("Proceso de unión y corte completo.")

El GeoDataFrame resultante de acueducto50_l.shp está vacío después del corte.
El GeoDataFrame resultante de canal50_l.shp está vacío después del corte.
El GeoDataFrame resultante de corriente_ag50_l.shp está vacío después del corte.
Archivo unido y cortado guardado en: CARTAS TOPOGRAFICAS\C09015_CUERPO_AGUA50_A_INEGI_SERIE_III.shp
Archivo unido y cortado guardado en: CARTAS TOPOGRAFICAS\C09015_CURVA_NIVEL50_L_INEGI_SERIE_III.shp
El GeoDataFrame resultante de linea_trans50_l.shp está vacío después del corte.
El GeoDataFrame resultante de canal50_a.shp está vacío después del corte.
El GeoDataFrame resultante de conducto50_l.shp está vacío después del corte.
Archivo unido y cortado guardado en: CARTAS TOPOGRAFICAS\C09015_VIA_FERREA50_L_INEGI_SERIE_III.shp
El GeoDataFrame resultante de aeropuerto50_a.shp está vacío después del corte.
Proceso de unión y corte completo.


In [7]:
LIMITE_ESTATAL = gpd.read_file("//Mycloudex2ultra/compartido/Base_Datos/INEGI/MGN/MGN_2023/mg_2023_integrado/conjunto_de_datos/00ent.shp").to_crs(epsg=6372)
print ("Cargado")
LOCALIDADES = gpd.read_file("//Mycloudex2ultra/compartido/Base_Datos/INEGI/MGN/MGN_2023/30_veracruzignaciodelallave/conjunto_de_datos/30l.shp").to_crs(epsg=6372)#CAMBIAR DE ACUERDO AL ESTADO
print ("Cargado")
LOCALIDADES_RURALES_PUNTOS = gpd.read_file("C:/SCINCE 2020/SCINCE VERACRUZ 2020/30_VER/cartografia/loc_rur.shp").to_crs(epsg=6372)#CAMBIAR DE ACUERDO AL ESTADO
print ("Cargado")
MANZANAS = gpd.read_file("//Mycloudex2ultra/compartido/Base_Datos/INEGI/MGN/MGN_2023/30_veracruzignaciodelallave/conjunto_de_datos/30m.shp").to_crs(epsg=6372)#CAMBIAR DE ACUERDO AL ESTADO
print ("Cargado")
print ("Carga completa")

Cargado
Cargado
Cargado
Cargado
Cargado
Cargado
Cargado
Carga completa


In [16]:
# Corte y guardado para LIMITE_MUNICIPAL
MUNICIPAL = gpd.overlay(LIMITE_MUNICIPAL, zona_corte, how='intersection')
if MUNICIPAL.empty:
    warnings.warn("Advertencia: El resultado de la intersección LIMITE_MUNICIPAL está vacío. No se generará un archivo de salida.")
nombre_archivo_MUNICIPAL = "MUNICIPAL.shp"
ruta_salida = f"C{cvegeo}_{os.path.splitext(nombre_archivo_MUNICIPAL)[0]}_INEGI_MGN_2023.shp"
if not MUNICIPAL.empty:
    MUNICIPAL.to_file(ruta_salida)

# Corte y guardado para LIMITE_ESTATAL
LIMITE_ESTATAL = gpd.overlay(LIMITE_ESTATAL, zona_corte, how='intersection')
if LIMITE_ESTATAL.empty:
    warnings.warn("Advertencia: El resultado de la intersección LIMITE_ESTATAL está vacío. No se generará un archivo de salida.")
nombre_archivo_ESTATAL = "LIMITE_ESTATAL.shp"
ruta_salida = f"C{cvegeo}_{os.path.splitext(nombre_archivo_ESTATAL)[0]}_INEGI_MGN_2023.shp"
if not LIMITE_ESTATAL.empty:
    LIMITE_ESTATAL.to_file(ruta_salida)

# Corte y guardado para LOCALIDADES
LOCALIDADES = gpd.overlay(LOCALIDADES, zona_corte, how='intersection')
if LOCALIDADES.empty:
    warnings.warn("Advertencia: El resultado de la intersección LOCALIDADES está vacío. No se generará un archivo de salida.")
nombre_archivo_LOCALIDADES = "LOCALIDADES.shp"
ruta_salida = f"C{cvegeo}_{os.path.splitext(nombre_archivo_LOCALIDADES)[0]}_INEGI_MGN_2023.shp"
if not LOCALIDADES.empty:
    LOCALIDADES.to_file(ruta_salida)
    
# Corte y guardado para LOCALIDADES_RURALES_PUNTOS
LOCALIDADES_RURALES_PUNTOS = gpd.overlay(LOCALIDADES_RURALES_PUNTOS, zona_corte, how='intersection')
if LOCALIDADES_RURALES_PUNTOS.empty:
    warnings.warn("Advertencia: El resultado de la intersección LOCALIDADES_RURALES_PUNTOS está vacío. No se generará un archivo de salida.")
nombre_archivo_LOCALIDADES_RURALES_PUNTOS = "LOCALIDADES_RURALES_PUNTOS.shp"
ruta_salida = f"C{cvegeo}_{os.path.splitext(nombre_archivo_LOCALIDADES_RURALES_PUNTOS)[0]}_INEGI_MGN_2023.shp"
if not LOCALIDADES_RURALES_PUNTOS.empty:
    LOCALIDADES_RURALES_PUNTOS.to_file(ruta_salida)

# Corte y guardado para MANZANAS
MANZANAS = gpd.overlay(MANZANAS, zona_corte, how='intersection')
if MANZANAS.empty:
    warnings.warn("Advertencia: El resultado de la intersección MANZANAS está vacío. No se generará un archivo de salida.")
nombre_archivo_MANZANAS = "MANZANAS.shp"
ruta_salida = f"C{cvegeo}_{os.path.splitext(nombre_archivo_MANZANAS)[0]}_INEGI_MGN_2023.shp"
if not MANZANAS.empty:
    MANZANAS.to_file(ruta_salida)

In [ ]:
ANP = gpd.read_file("C:/Users/andim/Documents/MAPA BASE/CAPAS BASE/SHAPE_ANPS_FEB2024/226_ANP_ITRF08_27022024.shp").to_crs(epsg=6372)
print ("Cargado")
VEGETACION = gpd.read_file("C:/Users/andim/Documents/MAPA BASE/CAPAS BASE/VEGETACION SERIE 7 CAPA PARA MAPAS BASE/VEGETACION SERIE 7 CAPA PARA MAPAS BASE/capa_vege_para_bases_inegi_s7_V2.shp").to_crs(epsg=6372)
print ("Cargado")
CALLES = gpd.read_file("C:/SCINCE 2020/SCINCE 2020 CDMX/09_CDMX/cartografia/eje_vial.shp").to_crs(epsg=6372)#CAMBIAR
print ("Cargado")
print ("Carga completa")

In [8]:
# Corte y guardado para ANP
ANP = gpd.overlay(ANP, zona_corte, how='intersection')
if ANP.empty:
    warnings.warn("Advertencia: El resultado de la intersección ANP está vacío. No se generará un archivo de salida.")
else:
    ANP = ANP[['NOMBRE']]
    ANP['Cve_Ent'] = cve_ent
    ANP['Cve_MunC'] = cvegeo
    nombre_archivo_ANP = "ANP.shp"
    ruta_salida = f"C{cvegeo}_{os.path.splitext(nombre_archivo_ANP)[0]}_CONAP_2024.shp"
    ANP.to_file(ruta_salida)

# Corte y guardado para VEGETACION
VEGETACION = gpd.overlay(VEGETACION, zona_corte, how='intersection')
if VEGETACION.empty:
    warnings.warn("Advertencia: El resultado de la intersección VEGETACION está vacío. No se generará un archivo de salida.")
else:
    VEGETACION = VEGETACION[['DESCRIPCIO']]
    VEGETACION['Cve_Ent'] = cve_ent
    VEGETACION['Cve_MunC'] = cvegeo
    nombre_archivo_VEGETACION = "VEGETACION.shp"
    ruta_salida = f"C{cvegeo}_{os.path.splitext(nombre_archivo_VEGETACION)[0]}_INEGI_MGN_2023.shp"
    ANP.to_file(ruta_salida)

print("Intersecciones completadas y archivos guardados.")

# Corte y guardado para CALLES
CALLES = gpd.overlay(CALLES, zona_corte, how='intersection')
if CALLES.empty:
    warnings.warn("Advertencia: El resultado de la intersección CALLES está vacío. No se generará un archivo de salida.")
else:
    CALLES['Cve_Ent'] = cve_ent
    CALLES['Cve_MunC'] = cvegeo
    nombre_archivo_CALLES = "RNC.shp"
    ruta_salida = f"C{cvegeo}_{os.path.splitext(nombre_archivo_CALLES)[0]}_INEGI_SCINCE_2020.shp"
    CALLES.to_file(ruta_salida)

C:\Users\ideapad Blue TDG\AppData\Local\Temp\ipykernel_21412\2813940015.py:16: UserWarning: Advertencia: El resultado de la intersección ANP está vacío. No se generará un archivo de salida.
  warnings.warn("Advertencia: El resultado de la intersección ANP está vacío. No se generará un archivo de salida.")
C:\Users\ideapad Blue TDG\anaconda3\envs\geo_env\lib\site-packages\geopandas\io\file.py:572: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


Intersecciones completadas y archivos guardados.


In [ ]:
RNC = gpd.read_file("//Mycloudex2ultra/compartido/Base_Datos/RED NACIONAL DE CAMINOS/RNC_2023/conjunto_de_datos/red_vial.shp").to_crs(epsg=6372)
print ("Cargado")
print ("Carga completa")

In [ ]:
RNC = gpd.read_file("//Mycloudex2ultra/compartido/Base_Datos/RED NACIONAL DE CAMINOS/RNC_2023/conjunto_de_datos/red_vial.shp").to_crs(epsg=6372)
print ("Cargado")

# Corte y guardado para RNC
RNC = gpd.overlay(RNC, zona_corte, how='intersection')
if RNC.empty:
    warnings.warn("Advertencia: El resultado de la intersección RNC está vacío. No se generará un archivo de salida.")
else:
    RNC = RNC[['TIPO_VIAL','NOMBRE', 'CODIGO', 'ADMINISTRA']]
    RNC['Cve_Ent'] = cve_ent
    RNC['Cve_MunC'] = cvegeo
    nombre_archivo_RNC = "RNC.shp"
    ruta_salida = f"C{cvegeo}_{os.path.splitext(nombre_archivo_RNC)[0]}_INEGI_RNC_2024.shp"
    RNC.to_file(ruta_salida)

In [14]:
RNC

,TIPO_VIAL,NOMBRE,CODIGO,ADMINISTRA,Cve_Ent,Cve_MunC
0,Viaducto,Río de la Piedad,N/A,N/A,09,09015
1,Avenida,Insurgentes Sur,N/A,N/A,09,09015
2,Viaducto,Río de la Piedad,N/A,N/A,09,09015
3,Calle,República de Brasil,N/A,N/A,09,09015
4,Circuito,Interior Revolución,N/A,N/A,09,09015
...,...,...,...,...,...,...
2032,Enlace,N/A,N/A,N/A,09,09015
2033,Circuito,Interior José Vasconcelos,N/A,N/A,09,09015
2034,Circuito,Interior José Vasconcelos,N/A,N/A,09,09015
2035,Circuito,Interior Melchor Ocampo,N/A,N/A,09,09015
